In [7]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
from utils import load_dataset

import time
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

In [9]:
df_creibles = load_dataset("data/junio23.csv", 15)

In [10]:
X = df_creibles.drop("fallecio", axis=1)
y = df_creibles["fallecio"]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =0, stratify=y)
print(X_train.shape)
print(X_test.shape)

(136086, 14)
(34022, 14)


In [15]:
lgbm_clf = joblib.load("models/lgbm_binary_auc_20200626-110602.pkl")

In [17]:
sigmoid = CalibratedClassifierCV(lgbm_clf, cv=2, method='sigmoid')

In [18]:
sigmoid.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=LGBMClassifier(bagging_fraction=0.99,
                                                     bagging_freq=28,
                                                     boosting_type='dart',
                                                     class_weight=None,
                                                     colsample_bytree=1.0,
                                                     feature_fraction=0.31431955825626834,
                                                     importance_type='split',
                                                     is_unbalance=True,
                                                     learning_rate=0.09056239592398911,
                                                     max_depth=75, metric='auc',
                                                     min_child_samples=20,
                                                     min_child_weight=7,
                                                     min_split_gain=0.0,
 

In [19]:
y_pred_cal = sigmoid.predict(X_test)
y_scores_cal = sigmoid.predict_proba(X_test)[:,1]

In [22]:
roc_auc_score(y_test, y_scores_cal)

0.8852884902567517

In [23]:
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = f'models/sigmoid_{timestr}.pkl'
joblib.dump(sigmoid, filename)

['models/sigmoid_20200626-110814.pkl']